In [0]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:13 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [88.1 kB]
Get:14 http://security.ubuntu.com/ubuntu bionic-security/restr

In [0]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup

In [0]:
import re

import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
lemmatizer = WordNetLemmatizer() 

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [0]:
pages = ["http://patft1.uspto.gov/netacgi/nph-Parser?Sect2=PTO1&Sect2=HITOFF&p=1&u=%2Fnetahtml%2FPTO%2Fsearch-bool.html&r=1&f=G&l=50&d=PALL&RefSrch=yes&Query=PN%2F9756055",
         "http://patft1.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&u=%2Fnetahtml%2FPTO%2Fsearch-adv.htm&r=2&p=1&f=G&l=50&d=PTXT&S1=python&OS=python&RS=python"]

In [0]:
final_docs = []
doc_names = []


for page in pages:
  driver.get(page)
  
  table_data=[]

  data = []
  content = driver.page_source
  soup = BeautifulSoup(content)
  for a in soup.findAll('table'):

      table_ele=a.find('tbody')


      table_data.append(re.split("\n",table_ele.text)) 

    
  for d in soup.findAll('body'):
      data.append(re.split("\n",d.text))
  
  docs_d=""
  docs=[]
  for j in table_data:
    for i in j:
      i = i.replace('/\s\s+/g', ' ')
      docs_d+=i+" "  
  docs.append(docs_d)

  docs_d=""

  for dt in data[0]:
      dt = dt.replace('/\s\s+/g', ' ')
      docs_d+=dt+" "  
  docs.append(docs_d)

  docs = ",".join(docs)
  con = word_tokenize(docs)
    
  doc1=""

  # docs = []

  # import re
  # for name in doc_names:
  #   doc1 = []
  #   dc1=""
  #   f = open(name, "r")
  #   content = f.read()
  #   con = re.split("\n|\t|''",content)
  #   f.close()
  for i in con:
      if i.isalnum():
        doc1+=((lemmatizer.lemmatize(i)))+" "

  final_docs.append(doc1)
  doc_names.append(driver.find_element_by_xpath("/html/body/table[2]/tbody/tr[1]/td[2]/b").text)

In [0]:
doc_names


['9,756,055', '10,623,958']

In [0]:
tfidf_vectorizer=TfidfVectorizer(use_idf=True)

# just send in all your docs here
fit_vectorizer=tfidf_vectorizer.fit(final_docs)
tfidf_vectorizer_vectors=fit_vectorizer.transform(final_docs)

In [0]:
first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[0]
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)


,tfidf
the,0.741420
access,0.236885
to,0.220734
resource,0.204300
of,0.171511
...,...
identical,0.000000
shot,0.000000
shipman,0.000000
hoyos,0.000000


In [0]:

from sklearn.metrics.pairwise import cosine_similarity

print("Cosine Similarity Scores are as follows")
display(pd.DataFrame(cosine_similarity(tfidf_vectorizer_vectors, tfidf_vectorizer_vectors),columns = doc_names,index = doc_names))

Cosine Similarity Scores are as follows


,"9,756,055","10,623,958"
"9,756,055",1.000000,0.760791
"10,623,958",0.760791,1.000000
